# ch00 — 数据准备

本 notebook 完成以下任务：
1. 下载/确认天池数据集
2. 加载并清洗原始数据
3. 基础 EDA：坏率分布、Vintage 分析、字段概况
4. 保存清洗后的数据供后续 notebook 使用

**数据集**：阿里云天池「金融风控-贷款违约预测」  
下载地址：https://tianchi.aliyun.com/competition/entrance/531830

下载后将 `train.csv` 和 `testA.csv` 放入 `../data/` 目录。

In [ ]:
import sys
sys.path.insert(0, '..')   # 使 creditrisk 包可导入

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from creditrisk.data import load_tianchi, train_val_split
from creditrisk.utils import data_overview, plot_vintage, Timer

plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

DATA_DIR = '../data'
print('环境准备完成')

## 1. 加载数据

In [ ]:
with Timer('加载数据集'):
    train, test = load_tianchi(DATA_DIR)

print('\n训练集前5行：')
train.head()

## 2. 数据概况

In [ ]:
overview = data_overview(train, target='isDefault')
print('\n缺失率 Top 10：')
overview.head(10)

In [ ]:
# 整体坏率
bad_rate = train['isDefault'].mean()
print(f'训练集坏率：{bad_rate:.4%}  ({train["isDefault"].sum():,} 条坏样本)')

## 3. 坏率分布分析

理解关键业务字段的坏率分布，是构建特征的起点。

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 8))

cat_cols = [
    ('grade',           '贷款等级'),
    ('term',            '贷款期限（月）'),
    ('homeOwnership',   '房屋所有权'),
    ('verificationStatus', '收入核验状态'),
    ('purpose',         '借款目的'),
    ('applicationType', '申请类型'),
]

for ax, (col, title) in zip(axes.flat, cat_cols):
    if col not in train.columns:
        ax.set_visible(False)
        continue
    grp = train.groupby(col)['isDefault'].agg(['mean','count'])
    grp = grp.sort_values('mean', ascending=False)
    bars = ax.bar(range(len(grp)), grp['mean'] * 100, color='steelblue', alpha=0.8)
    ax.set_xticks(range(len(grp)))
    ax.set_xticklabels(grp.index, rotation=30, ha='right', fontsize=8)
    ax.set_ylabel('坏率 (%)')
    ax.set_title(title)
    # 添加样本量标注
    for bar, cnt in zip(bars, grp['count']):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
                f'{cnt/1000:.0f}k', ha='center', fontsize=6)

plt.suptitle('各维度坏率分布', fontsize=14, y=1.01)
plt.tight_layout()
plt.show()

## 4. Vintage 分析

按放款年份观察坏率随时间的演变（此处用 `issueDate_year` 模拟账龄维度）。

In [ ]:
# 按放款年份统计坏率趋势
if 'issueDate_year' in train.columns:
    yearly = (train.groupby('issueDate_year')['isDefault']
              .agg(['mean','count'])
              .rename(columns={'mean':'bad_rate','count':'n_loans'}))
    print(yearly)

## 5. 数值特征分布

In [ ]:
num_cols = ['loanAmnt', 'interestRate', 'installment',
            'annualIncome', 'dti', 'ficoMean']
num_cols = [c for c in num_cols if c in train.columns]

fig, axes = plt.subplots(2, 3, figsize=(15, 7))
for ax, col in zip(axes.flat, num_cols):
    ax.hist(train[col].dropna().clip(
        train[col].quantile(0.01), train[col].quantile(0.99)
    ), bins=50, color='steelblue', alpha=0.7)
    ax.set_title(col)
    ax.set_ylabel('频次')
plt.tight_layout()
plt.show()

## 6. 保存清洗后数据

In [ ]:
import os
os.makedirs('../data/processed', exist_ok=True)

train.to_parquet('../data/processed/train_clean.parquet', index=False)
test.to_parquet('../data/processed/test_clean.parquet',  index=False)

print('数据已保存：')
print('  ../data/processed/train_clean.parquet')
print('  ../data/processed/test_clean.parquet')